# xarray-einstats for ArviZ

In [1]:
# needs dask_einops branch of xarray-einstats
import arviz as az
import numpy as np

from utils import ess_einstats

xarray-einstats versions of ess now!

Like with rhat, they work on DataArrays (and can be applied to Datasets with `.map`) of _any_ number of dimensions as long as there is a `chain` and a `draw` dimension (like in ArviZ, their actual order doesn't matter though). This page assumes you have already taken a look at the rhat experiments.

ess needs numba in addition to einops, dask continues to be an optional dependency.

## Compare to current performance

In [2]:
post = az.load_arviz_data("centered_eight").posterior
%timeit ess_einstats(post)
%timeit az.ess(post)

27.2 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.7 ms ± 534 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Same behaviour as with rhat, slower in very small dimension, already a bit faster for radon size datasets

In [3]:
post = az.load_arviz_data("radon").posterior
%timeit ess_einstats(post)
%timeit az.ess(post)

174 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
228 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Dask integration and further scaling properties

### Out of memory computation
`az.ess` is another one of the functions that already supports dask in ArviZ, so this would not make something
new possible with arviz, but I think that this approach of computation more integrated with xarray can
be applied to other functions and will be easier for dask integration with all stats and diagnostics.

The first step is showing that both `ess_einstats` defined here and `az.ess` can compute rhat on a ~15Gb dataset
from my 8Gb RAM laptop. Here I have used smaller chunks, following points 3 and 3 of [dask docs](https://docs.dask.org/en/stable/array-chunks.html) on chunking. `ess` is much more intensive in computational terms than `rhat`, so smaller chunks allow for more parallelization while still taking long enough to compute so that the overhead is negligible. Following point 4, I only chunk along the "parameter" dimension, leaving both chain and draw dimensions unchunked. Relevant quote from the page:

> For performance, a good choice of chunks follows the following rules:
>
> 1. A chunk should be small enough to fit comfortably in memory. We’ll have many chunks in memory at once
> 2. A chunk must be large enough so that computations on that chunk take significantly longer than the 1ms overhead per task that Dask scheduling incurs. A task should take longer than 100ms
> 3. Chunk sizes between 10MB-1GB are common, depending on the availability of RAM and the duration of computations
> 4. Chunks should align with the computation that you want to do.
> 5. Chunks should align with your storage, if applicable.

I did try bigger chunks like 3000 or 2000 and it was a bit slower, but I am not a Dask nor chunking expert (at least for now).

In [4]:
import dask
import time

data = dask.array.random.normal(size=(4, 1000, 600_000), chunks={2: 500})
post_dask = az.from_dict({"mu": data}, dims={"mu": ["obs_id"]}).posterior
data

dask.array<normal, shape=(4, 1000, 600000), dtype=float64, chunksize=(4, 1000, 500), chunktype=numpy.ndarray>

In [5]:
s = time.time()
aux = ess_einstats(post_dask, method="mean", dask="allowed")
aux = aux.compute()
print(time.time() - s)
aux

116.93356394767761


<xarray.Dataset>
Dimensions:  (obs_id: 600000)
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 ... 599995 599996 599997 599998 599999
Data variables:
    mu       (obs_id) float64 3.884e+03 3.497e+03 ... 3.72e+03 3.836e+03

In [6]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
s = time.time()
aux = az.ess(post_dask, method="mean").compute()
print(time.time() - s)
aux

arviz - WARNING - Shape validation failed: input_shape: (1, 1), minimum_shape: (chains=1, draws=4)


244.47589683532715


<xarray.Dataset>
Dimensions:  (obs_id: 600000)
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 ... 599995 599996 599997 599998 599999
Data variables:
    mu       (obs_id) float64 3.884e+03 3.497e+03 ... 3.72e+03 3.836e+03

We see that both work and give the same exact result. The current dask="parallelized" version is significantly slower, even with numba optimizations and even if the scipy part of the xarray-einstats implementation is also using dask="parallelized". Using dask="allowed" whenever possible allows dask to optimize much more the computation graph.

### Parallelized and optimized computation
Note: my idea was to run the comparisons below (except the first case) on `post_dask`, but it took very long and I
wasn't patient enough. So we compare with this dataset below with a subset of the chunks only. Doing that also has the side advantage of comparing numpy/in memory computation to dask parallelization and show it can even be useful on data that fits in memory if the computations are expensive.

This section therefore shows again the `ess_einstats` implementation but it is mostly an overview on the multiple
optimizations available in ArviZ right now side by side with this proposed change.

In [7]:
ds = post_dask.sel(obs_id=np.arange(18_000))
ds_in_mem = ds.compute()
ds.mu.data

dask.array<getitem, shape=(4, 1000, 18000), dtype=float64, chunksize=(4, 1000, 500), chunktype=numpy.ndarray>

#### `rhat_einstats`
##### In memory computation

In [8]:
s = time.time()
aux = ess_einstats(ds_in_mem, method="bulk")
print(time.time() - s)

21.188724279403687


(note that above we computed rhat on 1200 chunks instead of 36 chunks in 116 seconds, this should be an indicator on how good Dask is at parallelizing and optimizing the computational graph)

##### Distributed computing

In [9]:
s = time.time()
aux = ess_einstats(ds, method="bulk", dask="allowed").compute()
print(time.time() - s)

21.99575662612915


#### `az.rhat`
##### Dask + Numba

In [10]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
az.Numba.enable_numba()
s = time.time()
aux = az.ess(ds, method="bulk").compute()
print(time.time() - s)

22.816609382629395


##### Dask only

In [11]:
az.Dask.enable_dask(dask_kwargs={"dask": "parallelized"})
az.Numba.disable_numba()
s = time.time()
aux = az.ess(ds, method="bulk").compute()
print(time.time() - s)

21.899935245513916


##### Numba only (in memory computation)

In [12]:
az.Dask.disable_dask()
az.Numba.enable_numba()
s = time.time()
aux = az.ess(ds_in_mem, method="bulk")
print(time.time() - s)

21.501373291015625


##### No optimizations (in memory computation)

In [13]:
az.Dask.disable_dask()
az.Numba.disable_numba()
s = time.time()
aux = az.ess(ds_in_mem, method="bulk")
print(time.time() - s)

21.8496150970459


### Summary

There might be room for some further optimizations in `ess_einstats`, but this case isn't as clear cut as `rhat_einstats` to me. The function now _requires_ numba (instead of being a possible optimization avenue) and it does scale much better, but it doesn't really seem to have any advantage in small data (which will probably still be the most common input to ArviZ).

In my opinion, as it happened with `rhat_einstats`, is simpler and easier to understand. It doesn't need `_make_ufunc` nor `wrap_xarray_ufunc` wrappers, it doesn't have `if numba` statements (that also need duplicated testing because it is a manual 2nd implementation and sometimes it doesn't return the same result) and it doesn't need a dask activation class nor dask compatible alternatives.

We can try to look at improving the performance for small data if we want/need to (maybe for dashboards the 27ms vs 11ms is too much of a slowdown?) and it will probably be possible, my main focus on these experiments has been on scalability and on unifiying the code.

## Bonus: computation graph!
(needs to be right clicked and downloaded or opened on a new tab)

In [14]:
ess_einstats(ds, method="bulk", dask="allowed").mu.data.visualize()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.193831 to fit
